In [259]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

sns.set_theme()

# Load dataset

In [307]:
av_raw = pd.read_csv('archive/AviationData.csv',encoding='latin-1')

/var/folders/ty/9fdsc7tx2z99ntf3cyf1kjxh0000gn/T/ipykernel_33859/2738109947.py:1: DtypeWarning: Columns (6,7,28) have mixed types. Specify dtype option on import or set low_memory=False.
  av_raw = pd.read_csv('archive/AviationData.csv',encoding='latin-1')


In [260]:
pd.set_option('display.max_columns', None)
print(av_raw.shape)
av_raw.head()

(87282, 31)


,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,Injury.Severity,Aircraft.damage,Aircraft.Category,Registration.Number,Make,Model,Amateur.Built,Number.of.Engines,Engine.Type,FAR.Description,Schedule,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,Fatal(2),Destroyed,NaN,NC6404,Stinson,108-3,No,1.0,Reciprocating,NaN,NaN,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,Fatal(4),Destroyed,NaN,N5069P,Piper,PA24-180,No,1.0,Reciprocating,NaN,NaN,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,Fatal(3),Destroyed,NaN,N5142R,Cessna,172M,No,1.0,Reciprocating,NaN,NaN,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,Fatal(2),Destroyed,NaN,N1168J,Rockwell,112,No,1.0,Reciprocating,NaN,NaN,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,Fatal(1),Destroyed,NaN,N15NY,Cessna,501,No,NaN,NaN,NaN,NaN,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


In [266]:
av_raw[['Latitude','Longitude']].value_counts()

Latitude   Longitude  
332739N    1114342W       17
334118N    0112457W       14
61.174445  -149.996389    14
32.815556  -117.139444    14
61.213611  -149.844444    13
                          ..
31.151389  -81.391389      1
31.151111  -81.388055      1
31.15      -97.4           1
31.136944  -83.703889      1
9.956944   -84.139722      1
Length: 28849, dtype: int64

In [392]:
loc_df = av_raw.copy()
map_df = loc_df[['Latitude','Longitude']].astype(str)

map_data = map_df[~map_df.Latitude.str.contains("N",na=False)]

map_data = map_data[~map_data.Latitude.str.contains("S",na=False)]
map_data = map_data[~map_data.Longitude.str.contains("E",na=False)]
no_na = map_data[map_data != 'nan'].dropna()
print(no_na[0:5])
#df_filter = no_na.filter(regex="\d+\.\d+")


#map_data.loc[map_data[map_data != 'nan'].dropna().filter(regex="\d+\.\d+")]
#map_data = loc_df.filter(regex="\d+\.\d+")
#map_data[['Latitude','Longitude']].fillna(0)



       Latitude    Longitude
2     36.922223   -81.878056
5     42.445277   -70.758333
593   30.757778   -88.355555
3654  46.041111  -120.849722
6202      48.12    -113.8875


In [285]:
av_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87282 entries, 0 to 87281
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                87282 non-null  object 
 1   Investigation.Type      87282 non-null  object 
 2   Accident.Number         87282 non-null  object 
 3   Event.Date              87282 non-null  object 
 4   Location                87230 non-null  object 
 5   Country                 87056 non-null  object 
 6   Latitude                33073 non-null  object 
 7   Longitude               33064 non-null  object 
 8   Airport.Code            49322 non-null  object 
 9   Airport.Name            51955 non-null  object 
 10  Injury.Severity         86399 non-null  object 
 11  Aircraft.damage         84272 non-null  object 
 12  Aircraft.Category       30783 non-null  object 
 13  Registration.Number     85965 non-null  object 
 14  Make                    87219 non-null

# Data preprocessing

In [211]:
# Get all columns with empty values
av_raw.isna().any()

Event.Id                  False
Investigation.Type        False
Accident.Number           False
Event.Date                False
Location                   True
Country                    True
Latitude                   True
Longitude                  True
Airport.Code               True
Airport.Name               True
Injury.Severity            True
Aircraft.damage            True
Aircraft.Category          True
Registration.Number        True
Make                       True
Model                      True
Amateur.Built              True
Number.of.Engines          True
Engine.Type                True
FAR.Description            True
Schedule                   True
Purpose.of.flight          True
Air.carrier                True
Total.Fatal.Injuries       True
Total.Serious.Injuries     True
Total.Minor.Injuries       True
Total.Uninjured            True
Weather.Condition          True
Broad.phase.of.flight      True
Report.Status              True
Publication.Date           True
dtype: b

In [245]:
print(f"Empty lat : {av_raw['Latitude'].isna().sum()}")
print(f"Empty Location : {av_raw['Location'].isna().sum()}")
print(f"Empty airport codes: {av_raw['Airport.Code'].isna().sum()}")
print(f"Empty airport names : {av_raw['Airport.Name'].isna().sum()}")
print(f"Empty aircraft categories : {av_raw['Aircraft.Category'].isna().sum()}")
print(f"Empty aircraft engine : {av_raw['Number.of.Engines'].isna().sum()}")

Empty lat : 54209
Empty Location : 52
Empty airport codes: 37960
Empty airport names : 35327
Empty aircraft categories : 56499
Empty aircraft engine : 5749


In [246]:
import geopandas as gp 
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="av_app")

#geoencode locations
av_raw['Location'].fillna("Alaska",inplace=True)
av_raw['Country'].fillna("Alaska",inplace=True)

empty_lat_lon = av_raw.loc[av_raw['Latitude'].isna(),['Event.Id','Location','Latitude','Country','Longitude']]
print(empty_lat_lon.shape)

(54209, 5)


In [ ]:

# geocode the location column 
#create empty dataframe to store the geocoded data
from socket import timeout
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm
tqdm.pandas()
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

encode = pd.DataFrame(columns=['Event.Id','Location','Latitude','Longitude'])
encode['Event.Id'] = empty_lat_lon['Event.Id']
encode['Location'] = empty_lat_lon['Location'].progress_apply(geocode)
# extract lat and lon from geocoded locations
encode['Latitude'] = encode['Location'].apply(lambda x: x.latitude if x != None else None)
encode['Longitude'] = encode['Location'].apply(lambda x: x.longitude if x != None else None)

print(encode.head())
# join encode with av_raw on id 
av_raw_merged = av_raw.merge(encode,on='Event.Id',how="left")

av_raw_merged['Longitude_y'] = av_raw_merged['Longitude_y'].fillna(av_raw_merged['Longitude_x'])
av_raw_merged['Latitude_y'] = av_raw_merged['Latitude_y'].fillna(av_raw_merged['Latitude_x'])

av_raw_merged['Location_y'] = av_raw_merged['Location_y'].fillna(av_raw_merged['Location_x'])

av_raw_merged.drop(["Longitude_x"], inplace=True, axis=1)
av_raw_merged.drop(["Latitude_x"], inplace=True, axis=1)
av_raw_merged.drop(["Location_x"], inplace=True, axis=1)
av_raw_merged.rename(columns={'Longitude_y':'Longitude','Latitude_y':'Latitude','Location_y':'Location'},inplace=True)

print(av_raw_merged.info())
av_raw = av_raw_merged


In [253]:
import datetime
#remove Event.ID
av_raw.drop(columns=["Event.Id","Accident.Number","Registration.Number","Schedule","Injury.Severity"],inplace=True,errors='ignore');

#TODO combine aircraft make and model into one feature and encode

av_raw['FAR.Description'].fillna(value='unknown',inplace=True)

# TODO Fill NA Lat/Lon with corresponding locations 

# Fill empty aircraft category with 'unknown'
av_raw['Aircraft.Category'].fillna(value="unknown",inplace=True)

#group events by year
av_raw['year'] = pd.DatetimeIndex(av_raw['Event.Date']).year
av_raw['month'] = pd.DatetimeIndex(av_raw['Event.Date']).month
# make year of type numeric
av_raw.head()

av_raw.sort_values(by=['year'])

# get the report status after 2020
reps_08 = av_raw[av_raw['year'] < 2008]['Report.Status']

reps_nlp = av_raw[av_raw['year'] > 2008]['Report.Status']

# Report status before 2009 do not have detailed information on cause of accident to analyze - reports after 2009 are more interesting to analyze using NLP methods to group accidents by cause
print(reps_08.value_counts())
print(reps_nlp.value_counts())

print(av_raw.isna().any())

Probable Cause    61754
Foreign            1999
Factual             145
Preliminary          15
Name: Report.Status, dtype: int64
<br /><br />                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              163
The pilot's failure to maintain directional control during the landing roll.                                                                                                                                                                                                                                            

In [ ]:
# Load the example flights dataset and convert to long-form

accidents = av_raw.pivot("month", "year", "Total.Fatal.Injuries")

# Draw a heatmap with the numeric values in each cell
f, ax = plt.subplots(figsize=(9, 6))
sns.heatmap(av_raw, annot=True, fmt="d", linewidths=.5, ax=ax)

# Analysis
# Questions
- What is the phase of flight where most accidents happened
- What is the model with the most accidents
- What is the airport with the most accidents
- What weather conditions occured during most accidents?

# Ideas
- Do NLP analysis on report status
- Create a map with lat/lon points of accidents

In [ ]:
# Create map with 

# What is the phase of flight where most accidents happened?

In [263]:

# Analyse phase of flight

print(av_raw['Broad.phase.of.flight'].value_counts())

# make plot a little bigger
plt.figure(figsize=(20,10))
sns.histplot(av_raw,x='year',y='Broad.phase.of.flight',bins=30, discrete=(True, True), log_scale=(False, False),)


Landing        15428
Takeoff        12493
Cruise         10269
Maneuvering     8144
Approach        6546
Climb           2034
Taxi            1958
Descent         1887
Go-around       1353
Standing         945
Unknown          548
Other            119
Name: Broad.phase.of.flight, dtype: int64


ValueError: Could not interpret value `year` for parameter `x`

<Figure size 1440x720 with 0 Axes>

In [53]:
av_raw['Injury.Severity'].value_counts()

Non-Fatal     66561
Fatal(1)       6167
Fatal          4959
Fatal(2)       3711
Incident       2219
              ...  
Fatal(228)        1
Fatal(229)        1
Fatal(80)         1
Fatal(217)        1
Fatal(72)         1
Name: Injury.Severity, Length: 107, dtype: int64

# NLP on report status

In [26]:
av_raw['Report.Status'].value_counts()

Probable Cause                                                                                                                                                       61754
Foreign                                                                                                                                                               1999
<br /><br />                                                                                                                                                           167
Factual                                                                                                                                                                145
The pilot's failure to maintain directional control during the landing roll.                                                                                            56
                                                                                                                                                 

# Unsupervised learning
- Use clustering algorithms to explore dataset and find interesting patterns of "accident groups" => then analyze each cluster seperately
- Anomaly detection

In [50]:
from sklearn.preprocessing import OneHotEncoder
# get all non-numerical columns
categorical_cols = av_raw.select_dtypes(exclude=['number']).columns
enc = OneHotEncoder(handle_unknown='ignore')

for col in categorical_cols:
    enc.fit(col)


KeyError: "None of [Index(['I'], dtype='object')] are in the [columns]"

In [ ]:
from sklearn.cluster import KMeans
k = 5
kmeans = KMeans(n_clusters=k, random_state=0)
y_pred = kmeans.fit_predict(av_raw)